In [1]:
import serial
import time
import pygame
import sys

In [2]:
pygame.init()
pygame.joystick.init()

joystick = pygame.joystick.Joystick(0)  
joystick.init()

print("Using joystick: ", joystick.get_name())

('Using joystick: ', 'Harmonix Xbox 360 Controller')


In [3]:
def clear():
    s = '\r' + ' '*1000
    sys.stdout.write(s)
    sys.stdout.flush()

In [4]:
upDownAxis = 1
leftRightAxis = 0
aButton = 0
bButton = 1
xButton = 2
yButton = 3
backButton = 6
startButton = 7
xboxButton = 8

cycle = 0

In [5]:
from bitstring import BitArray
def writeThenRead(val):
    global cycle
    s = "\r"
    cycle += 1
    if cycle % 4 == 0:
        s += '| '
    if cycle % 4 == 1:
        s += '/ '
    if cycle % 4 == 2:
        s += '- '
    if cycle % 4 == 3:
        s += '\ '
    bitarr = BitArray(uint=val, length=8).bin
    s += "wrote: {} read: ".format(bitarr)
    ser.write(chr(val))
    ser.flush()
    while ser.inWaiting():
        line = ser.readline()
        s += line.rstrip() + " " 
    
    clear()
    sys.stdout.write(s)
    sys.stdout.flush()

In [ ]:
ser = serial.Serial('/dev/ttyACM0')
running = True
while running:
    pygame.event.pump()
    if not joystick.get_button(xboxButton):
        running = False
        
running = True
s = "Starting..."
while running:
#     sys.stdout.write("\r                                                          ")
#     sys.stdout.write("\r{}".format(s))
#     sys.stdout.flush()    
    s = ""

    pygame.event.pump()

    # Special
    if joystick.get_button(xboxButton):
        print("Quitting...\n")
        running = False
        continue
        
    s = "Pressed: ";
    if joystick.get_button(startButton):
        writeThenRead(int('11000000', 2))
        s += "Start "
        time.sleep(0.1)
        continue
    if joystick.get_button(backButton):
        writeThenRead(int('00110000', 2))
        s += "Back "
        time.sleep(0.1)
        continue
    
    byte = 0
    # Up/Down, Left/Right
    axis0 = joystick.get_axis( upDownAxis )
    if axis0 > 0:
        byte = byte | (1 << 7)
        s += "Down "
    if axis0 < 0:
        byte = byte | (1 << 6)
        s += "Up "
    axis1 = joystick.get_axis( leftRightAxis )
    if axis1 > 0:
        byte = byte | (1 << 5)
        s += "Right "
    if axis1 < 0:
        byte = byte | (1 << 4)
        s += "Left "
            
    # Buttons
    if joystick.get_button(aButton):
        byte = byte | (1 << 3)
        s += "A "
    if joystick.get_button(bButton):
        byte = byte | (1 << 2)
        s += "B "
    if joystick.get_button(xButton):
        byte = byte | (1 << 1)
        s += "X "
    if joystick.get_button(yButton):
        byte = byte | 1
        s += "Y "
    
    s += bin(byte)
    writeThenRead(byte)
    time.sleep(0.1)

if ser is not None:
    ser.close()

- wrote: 00000000 read: buttons: All off                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [7]:
tmp = 0
for i in range(8):
    tmp = 0 | (1 << i)
    print(bin(tmp), BitArray(uint=tmp, length=8).bin)

('0b1', '00000001')
('0b10', '00000010')
('0b100', '00000100')
('0b1000', '00001000')
('0b10000', '00010000')
('0b100000', '00100000')
('0b1000000', '01000000')
('0b10000000', '10000000')
